In [71]:
# Rename me from Pickle_Data_A1a to iPythonExample_stepA
workarea = "/Users/dork/Home/GithubRepo/"
file_storage = workarea+"Files/"

dataset = "2013_recoA1a"
#dataset = "ic86_goodA_p_le"
#dataset = "ic86_goodA_i_le"

In [72]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor,MultiOutputClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score
from scipy.stats import pearsonr

from sklearn import datasets
from sklearn.model_selection  import GridSearchCV

import tables
import pylab as pl

import math

import pickle

import EmilysTools
from EmilysTools import Grabber, DataGrabber, llh_check, PassedDataG1Cuts, PassedDataG2Cuts, anti_llh_check, AntiPassedDataG1Cuts

In [73]:


it_orig_qc_list = ['BetaCutPassed','IceTopMaxSignalAbove6','IceTopMaxSignalInside','IceTopNeighbourMaxSignalAbove4','IceTop_StandardFilter','IceTop_reco_succeeded','Laputop_FractionContainment','StationDensity_passed']
ii_orig_qc_list = ['StochRecoSucceeded','MilliNCascAbove2','MilliQtotRatio','MilliRloglBelow2']

g1_it_qc_list = ['IceTopMaxSignalAbove6','IceTopMaxSignalInside','IceTopNeighbourMaxSignalAbove4','IceTop_StandardFilter','StationDensity_passed']
g1_ii_qc_list = ['good_ElossReco_IT_mpeLLH_itSEEDMilliNCascAbove2','good_ElossReco_IT_mpeLLH_itSEEDMilliQtotRatio','good_ElossReco_IT_mpeLLH_itSEEDMilliRloglBelow2','good_ElossReco_IT_mpeLLH_itSEEDStochRecoSucceeded']

g2_it_qc_list = ['IceTop_StandardFilter','StationDensity_passed']
g2_ii_qc_list = ['good_ElossReco_II_speLLH_mpeSEEDMilliNCascAbove2','good_ElossReco_II_speLLH_mpeSEEDMilliQtotRatio','good_ElossReco_II_speLLH_mpeSEEDMilliRloglBelow2','good_ElossReco_II_speLLH_mpeSEEDStochRecoSucceeded']


In [74]:
test_file = DataGrabber(file_storage+dataset+'.hdff')
#test_file = Grabber(file_storage+dataset+'.hdff')

In [75]:
for key in test_file.keys():
    print(key)

good_ElossReco_IT_mpeLLH_itSEEDMilliNCascAbove2
good_ElossReco_IT_mpeLLH_itSEEDMilliQtotRatio
good_ElossReco_IT_mpeLLH_itSEEDMilliRloglBelow2
good_ElossReco_IT_mpeLLH_itSEEDStochRecoSucceeded
good_ElossReco_II_speLLH_mpeSEEDMilliNCascAbove2
good_ElossReco_II_speLLH_mpeSEEDMilliQtotRatio
good_ElossReco_II_speLLH_mpeSEEDMilliRloglBelow2
good_ElossReco_II_speLLH_mpeSEEDStochRecoSucceeded
BetaCutPassed
IceTopMaxSignalAbove6
IceTopMaxSignalInside
IceTopNeighbourMaxSignalAbove4
IceTop_StandardFilter
IceTop_reco_succeeded
Laputop_FractionContainment
StationDensity_passed
MilliNCascAbove2
MilliQtotRatio
MilliRloglBelow2
NCh_CoincLaputopCleanedPulsesAbove7
StochRecoSucceeded
IT_mpeLLH_itSEED_x
IT_mpeLLH_itSEED_y
IT_mpeLLH_itSEED_z
IT_mpeLLH_itSEED_time
IT_mpeLLH_itSEED_zenith
IT_mpeLLH_itSEED_azimuth
IT_mpeLLH_itSEED_energy
IT_mpeLLH_itSEED_speed
IT_mpeLLH_itSEED_length
IT_mpeLLH_itSEED_pdg_encoding
IT_mpeLLH_itSEED_type
IT_mpeLLH_itSEED_shape
IT_mpeLLH_itSEED_location
IT_mpeLLH_itSEED_fit_stat

In [76]:
my_g1_data = llh_check(test_file,'Good_COMBO_IT_value')   


In [77]:
my_g1_data = PassedDataG1Cuts(my_g1_data,g1_it_qc_list,'Good_COMBO_IT_value')
my_g1_data = PassedDataG1Cuts(my_g1_data,g1_ii_qc_list,'Good_COMBO_IT_value') 
my_g1_data =(my_g1_data.loc[(my_g1_data['IT_mpeLLH_itSEEDParams_Beta'] <9.5)])
my_g1_data =(my_g1_data.loc[(my_g1_data['IT_mpeLLH_itSEEDParams_Beta'] >1.4)])
my_g1_data =(my_g1_data.loc[(my_g1_data['IT_mpeLLH_itSEEDParams_Log10_S125'] >0.)])

my_g1_data =(my_g1_data.loc[(np.degrees(my_g1_data['IT_mpeLLH_itSEED_zenith']) <35.)])
my_g1_data['IT_mpeLLH_itSEED_coszen']=np.cos(my_g1_data['IT_mpeLLH_itSEED_zenith'])
my_g1_data = my_g1_data.dropna(subset = ['good_ElossReco_IT_mpeLLH_itSEEDStoch_Reco_EM_eloss_1500'])
#my_g1_data =(my_g1_data.loc[((my_g1_data['II_speLLH_mpeSEEDParams_Log10_S125'] ==0.0))])
#my_g1_data =(my_g1_data.loc[((my_g1_data['II_speLLH_mpeSEEDParams_Beta'] ==0.0))])
print(len(my_g1_data))

75936


In [78]:
my_g2_data = llh_check(test_file,'Good_COMBO_II_value')   
my_g2_data = PassedDataG2Cuts(my_g2_data,g2_it_qc_list,'Good_COMBO_II_value')
my_g2_data = PassedDataG2Cuts(my_g2_data,g2_ii_qc_list,'Good_COMBO_II_value') 
my_g2_data =(my_g2_data.loc[(my_g2_data['II_speLLH_mpeSEEDParams_Beta'] <9.5)])
my_g2_data =(my_g2_data.loc[(my_g2_data['II_speLLH_mpeSEEDParams_Beta'] >1.4)])
my_g2_data =(my_g2_data.loc[(my_g2_data['II_speLLH_mpeSEEDParams_Log10_S125'] >0.)])
my_g2_data =(my_g2_data.loc[(np.degrees(my_g2_data['II_speLLH_mpeSEED_zenith']) <90.)])
my_g2_data =(my_g2_data.loc[(my_g2_data['G2diff_value'] <100.)])
my_g2_data['II_speLLH_mpeSEED_coszen']=np.cos(my_g2_data['II_speLLH_mpeSEED_zenith'])
print(len(my_g2_data))

30522


In [79]:
my_data = llh_check(test_file,'Good_COMBO_IT_value')   
my_data = PassedDataG1Cuts(my_data,g1_it_qc_list,'Good_COMBO_IT_value')
my_data = PassedDataG1Cuts(my_data,g1_ii_qc_list,'Good_COMBO_IT_value') 
my_data =(my_g1_data.loc[(my_data['IT_mpeLLH_itSEEDParams_Beta'] <9.5)])
my_data =(my_g1_data.loc[(my_data['IT_mpeLLH_itSEEDParams_Beta'] >1.4)])
my_data['IT_mpeLLH_itSEED_coszen']=np.cos(my_data['IT_mpeLLH_itSEED_zenith'])
my_data = my_data.dropna(subset = ['good_ElossReco_IT_mpeLLH_itSEEDStoch_Reco_EM_eloss_1500'])
my_data =(my_data.loc[((my_data['II_speLLH_mpeSEEDParams_Log10_S125'] ==0.0))])
my_data =(my_data.loc[((my_data['II_speLLH_mpeSEEDParams_Beta'] ==0.0))])
my_data = pd.concat([my_data,my_g2_data],axis=1)
print(len(my_data))

101294


In [80]:
og_all = llh_check(test_file,'Good_Laputop_value')   
og_all = llh_check(og_all,'Good_MPE_value') 
og_all = PassedDataG1Cuts(og_all,ii_orig_qc_list,'Good_MPE_value') 
og_all = PassedDataG1Cuts(og_all,it_orig_qc_list,'Good_Laputop_value') 
og_all =(og_all.loc[(og_all['LaputopParams_beta'] <9.5)])
og_all =(og_all.loc[(og_all['LaputopParams_beta'] >1.4)])
og_all =(og_all.loc[(np.log10(og_all['LaputopParams_s125']) >0.)])
og_all['Laputop_coszen']=np.cos(og_all['Laputop_zenith'])
print(len(og_all))
# matches with DataPlotter3.py when feed group arguments <1 True 86p>

101585


In [81]:
pickle.dump( my_g1_data, open( file_storage+"stepA_3Dg1_"+dataset+".pkl", "wb" ) )

In [82]:
pickle.dump( my_g2_data, open( file_storage+"stepA_3Dg2_"+dataset+".pkl", "wb" ) )

In [83]:
pickle.dump( my_data, open( file_storage+"stepA_3Dall_"+dataset+".pkl", "wb" ) )

In [84]:
pickle.dump( og_all, open( file_storage+"stepA_OGall_"+dataset+".pkl", "wb" ) )